In [0]:
# Célula 1: Leitura da Camada Bronze
from pyspark.sql.functions import col, round

# Lemos do catálogo que criamos
df = spark.read.table("portfolio_credit.german_credit_bronze")

print("Dados carregados. Vamos transformar!")

In [0]:
# Célula 2: Feature Creation (Criando novas inteligências)

# Vamos criar uma estimativa do valor da parcela mensal: Valor Total / Meses
df_silver = df.withColumn("monthly_installment", 
                          round(col("credit_amount") / col("duration"), 2))

# Selecionamos apenas as colunas que vamos usar no modelo (excluindo algumas irrelevantes para simplificar)
# Vamos focar nas mais impactantes que vimos na EDA
cols_to_keep = [
    "label",               # Nossa meta (0 ou 1)
    "checking_status",     # Importante (Vimos na EDA)
    "duration",            # Importante (Vimos na EDA)
    "credit_history",      # Histórico conta muito
    "purpose",             # Motivo do empréstimo
    "credit_amount",       # Valor total
    "monthly_installment", # A NOVA variável que criamos
    "age",                 # Idade
    "housing"              # Se tem casa própria
]

df_selected = df_silver.select(cols_to_keep)

display(df_selected.head(5))

In [0]:
# Célula 3: Encoding (Transformando Categorias em Números)
from pyspark.ml.feature import StringIndexer

# Lista das colunas que são texto e precisam virar número
cat_cols = ["checking_status", "credit_history", "purpose", "housing"]

# Loop para transformar cada uma
# Criamos colunas novas com sufixo "_ix" (Ex: purpose -> purpose_ix)
for column in cat_cols:
    indexer = StringIndexer(inputCol=column, outputCol=column + "_ix")
    # Fit e Transform aprendem o mapeamento e aplicam
    df_selected = indexer.fit(df_selected).transform(df_selected)

# Agora removemos as colunas de texto originais, pois o modelo só quer as numéricas
df_final_silver = df_selected.drop(*cat_cols)

print("Transformação concluída. Veja que agora tudo é número:")
display(df_final_silver)

In [0]:
# Célula 4: Salvar na Camada Silver
table_name = "portfolio_credit.german_credit_silver"

df_final_silver.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"Tabela Silver salva: {table_name}")
print(f"Total de Colunas para o Modelo: {len(df_final_silver.columns)}")